In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import random

In [59]:
DIRECTORY = r"C:\\Users\\User\\Machine-Learning-main\\Deep Learning\\CNN\\Covid\\train"

data = []

CATEGORIES = ["Covid","Normal","Viral Pneumonia"]

for category in CATEGORIES:
    folder = os.path.join(DIRECTORY,category)
  
    label = CATEGORIES.index(category)
    
    for img in os.listdir(folder):
        img = os.path.join(folder,img)
        
        img_arr = cv2.imread(img)
        img_arr = cv2.resize(img_arr, (100,100))
        #plt.imshow(img_arr)
        #plt.show()
        
        data.append([img_arr,label])
      

In [54]:
#Data Preprocessing
IMG_SIZE = 224
BATCH_SIZE = 64
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.1 
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.1 
)

train_datagen = train_datagen.flow_from_directory(
    DIRECTORY,
    target_size= (IMG_SIZE, IMG_SIZE),
    batch_size= BATCH_SIZE,
    subset= "training"
)

test_datagen = test_datagen.flow_from_directory(
    DIRECTORY,
    target_size= (IMG_SIZE, IMG_SIZE),
    batch_size= BATCH_SIZE,
    subset= "validation"
)

Found 226 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [55]:
train_datagen.class_indices

{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}

In [56]:
#Model Generate
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(filters=32,padding="same",strides=2,kernel_size=3,activation="relu",input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(MaxPooling2D(pool_size=2,strides=2))

model.add(Flatten())
model.add(Dense(3,activation = "softmax"))

In [61]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_datagen,epochs=10,validation_data=test_datagen)

Epoch 1/10
4/4 [==============================] - 18s 4s/step - loss: 0.6037 - accuracy: 0.7788 - val_loss: 0.9648 - val_accuracy: 0.6400
Epoch 2/10
4/4 [==============================] - 16s 5s/step - loss: 0.4421 - accuracy: 0.8363 - val_loss: 0.7302 - val_accuracy: 0.6400
Epoch 3/10
4/4 [==============================] - 16s 4s/step - loss: 0.3761 - accuracy: 0.8451 - val_loss: 0.7020 - val_accuracy: 0.7600
Epoch 4/10
4/4 [==============================] - 15s 4s/step - loss: 0.3385 - accuracy: 0.8850 - val_loss: 0.6634 - val_accuracy: 0.7600
Epoch 5/10
4/4 [==============================] - 16s 4s/step - loss: 0.3267 - accuracy: 0.8584 - val_loss: 0.4947 - val_accuracy: 0.8800
Epoch 6/10
4/4 [==============================] - 15s 4s/step - loss: 0.2749 - accuracy: 0.8850 - val_loss: 0.8039 - val_accuracy: 0.8000
Epoch 7/10
4/4 [==============================] - 15s 4s/step - loss: 0.2945 - accuracy: 0.8894 - val_loss: 0.4238 - val_accuracy: 0.8400
Epoch 8/10
4/4 [==================

In [62]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 100352)            0         
                                                                 
 dense_6 (Dense)             (None, 3)                 301059    
                                                                 
Total params: 301,955
Trainable params: 301,955
Non-trainable params: 0
_________________________________________________________________


In [63]:
np.array(history.history['accuracy']).mean()

0.8641593039035798

In [75]:
from keras.preprocessing import image
path = "C:\\Users\\User\\Machine-Learning-main\\Deep Learning\\CNN\\Covid\\net\\"
img = path + "covid-1.jpg"

img_pred = tf.keras.utils.load_img(img,target_size=(IMG_SIZE,IMG_SIZE))
img_pred = tf.keras.utils.img_to_array(img_pred)
img_pred = img_pred/255
img_pred = np.expand_dims(img_pred, axis=0)
result = model.predict(img_pred)

print(CATEGORIES[np.argmax(result)])

1/1 [==============================] - 0s 25ms/step
Covid
